In [1]:
import matplotlib
#matplotlib.use('Agg')
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

In [2]:
import numpy as np
import h5py
#from chainconsumer import ChainConsumer
from corner import corner

In [3]:
obs = 'ds'

if obs == 'wp':
    idx = 0
else:
    idx = 1

In [4]:
fname = '/u/ki/swmclau2/des/SherlockPearceMCMC/TAB_HOD_wp_ds_rmin_None_TAB.hdf5'

In [5]:
f = h5py.File(fname, 'r')

IOError: Unable to open file (unable to open file: name = '/u/ki/swmclau2/des/SherlockPearceMCMC/TAB_HOD_wp_ds_rmin_None_TAB.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
f.attrs.keys()

In [ ]:
from ast import literal_eval

In [ ]:
from pearce.emulator import LemonPepperWet
tf = f.attrs['training_file']
fixed_params = literal_eval(f.attrs['fixed_params'])
emu_hps = literal_eval(f.attrs['emu_hps'])

In [ ]:
tf

In [ ]:
emu = LemonPepperWet('/u/ki/swmclau2/des/%s_zheng07/PearceDsCosmo.hdf5'%obs, fixed_params = fixed_params, hyperparams = emu_hps)

In [ ]:
emu.obs

In [ ]:
chain_pnames = f.attrs['param_names']

In [ ]:
n_walkers = f.attrs['nwalkers']

In [ ]:
chain = f['chain'][()]#[n_walkers*1000:n_walkers*6000]

In [ ]:
n_params = chain.shape[1] if len(chain.shape) > 1 else 1

In [ ]:
n_burn = 49000
chain = chain[n_walkers*n_burn:]#, :]

In [ ]:
chain = chain[np.all(chain!=0.0, axis = 1), :]

In [ ]:
print chain.shape, chain.shape[0]/n_walkers

In [ ]:
MAP = chain.mean(axis = 0)
print MAP

In [ ]:
chain_pnames

In [ ]:
low_om_idxs = chain[:,1] < 0.116
med_om_idxs = np.logical_and(chain[:,1] < 0.117, chain[:,1]>0.113)


low_om_chain = chain[low_om_idxs]
high_om_chain = chain[~low_om_idxs]
med_om_chain = chain[med_om_idxs]

In [ ]:
MAP_dict = dict(zip(emu.get_param_names(), MAP))
low_om_MAP_dict = dict(zip(emu.get_param_names(), low_om_chain.mean(axis =0)))
high_om_MAP_dict = dict(zip(emu.get_param_names(), high_om_chain.mean(axis=0)))
med_om_MAP_dict = dict(zip(emu.get_param_names(), med_om_chain.mean(axis=0)))

In [ ]:
MAP_dict

In [ ]:
fixed_params = eval(f.attrs['chain_fixed_params'])

In [ ]:
MAP_dict.update(fixed_params)

In [ ]:
MAP_dict

In [ ]:
MAP_pred = emu.emulate_wrt_r(MAP_dict, r_bin_centers=emu.scale_bin_centers).squeeze()
#low_om_pred = emu.emulate_wrt_r(low_om_MAP_dict)[0]
#high_om_pred = emu.emulate_wrt_r(high_om_MAP_dict)[0]

In [ ]:
MAP_pred

In [ ]:
param_names = [r'$\Omega_b h^2$', r'$\Omega_c h^2$', r'$w_0$', r'$n_s$', r'$\ln(10A_s)$', r'$H_0$', r'$N_{eff}$',\
               r'$\log(M_0)$', r'$\sigma_{\log M }$',r'$\log(M_1)$',r'$\alpha$']
               # r'$\rho_{sat}$',r'$\rho_{cen}$', r'$\log(M_1)$',r'$\log(M_0)$', r'$\sigma_{\log M }$',r'$\alpha$' ]

In [ ]:
hod_param_names = [r'$\log(M_0)$',r'$\sigma_{\log M }$',r'$\log(M_1)$' ,r'$\alpha$' ]
cosmo_param_names = [r'$\Omega_b h^2$', r'$\Omega_c h^2$', r'$w_0$', r'$n_s$', r'$\ln(10A_s)$', r'$H_0$', r'$N_{eff}$' ]

In [ ]:
hod_idxs = np.array(range(len(cosmo_param_names), len(cosmo_param_names)+len(hod_param_names)))
cosmo_idxs = np.array(range(len(cosmo_param_names)))

In [ ]:
if chain.shape[1] == 7:
    cosmo_chain = chain
else:
    hod_chain = chain[:,-4:]#[:, hod_idxs]
    cosmo_chain = chain[:,:-4]#[:, cosmo_idxs]

In [ ]:
from pearce.mocks import cat_dict
cosmo_params = {}

In [ ]:
cosmo_params['boxno'] = 1
cosmo_params['realization'] = 0

In [ ]:
cat = cat_dict['testbox'](**cosmo_params)#construct the specified catalog!

In [ ]:
cpv = cat._get_cosmo_param_names_vals()

cat_val_dict =  {key: val for key, val in zip(cpv[0], cpv[1])}

In [ ]:
import yaml
with open('/u/ki/swmclau2/Git/pearce/bin/mcmc/config/HOD_wp_ds_rmin_None_HOD.yaml') as yamlfile:
    cfg = yaml.load(yamlfile)
    sim_cfg = cfg['data']['sim']
    obs_cfg = cfg['data']['obs']

In [ ]:
r_bins = obs_cfg['rbins']

In [ ]:
true_param_dict = cat_val_dict.copy()
hod_params = {'alpha': 1.083, 'conc_gal_bias': 1.0, 'logM0': 13.2,'logM1': 14.2, 'sigma_logM': 0.2}#sim_info['hod_params']

#hod_params['mean_occupation_centrals_assembias_corr1'] =0.0
#hod_params['mean_occupation_satellites_assembias_corr1'] =0.0

for hp, hv in hod_params.iteritems():
    if hp == 'logMmin':
        continue
    true_param_dict[hp] = hv

true_param_dict['conc_gal_bias'] = 1.0
true_pred = emu.emulate_wrt_r(true_param_dict)[0]

In [ ]:
f['data']

In [ ]:
true_data = f['data'][idx]

In [ ]:
true_data

In [ ]:
rbc = emu.scale_bin_centers#[-len(emu.scale_bin_centers):]

In [ ]:
if obs == 'wp':
    cov = f['cov'][:len(r_bins)-1][:, :len(r_bins)-1]
else:
    cov = f['cov'][len(r_bins)-1:][:, len(r_bins)-1:]

yerr = np.sqrt(np.diag(cov))[-len(rbc):]

In [ ]:
yerr

In [ ]:
yerr/true_data[-len(emu.scale_bin_centers):]

In [ ]:
plt.imshow(np.log10(cov) )

In [ ]:
np.sqrt(np.diag(cov))

In [ ]:
np.sqrt(np.diag(f['cov'][()]))#.reshape((2,18))

In [ ]:
bounds = {pname: emu.get_param_bounds(pname) for pname in emu.get_param_names()[:-4]}

In [ ]:
bounds

In [ ]:
fig = plt.figure(figsize = (10,7))
plt.errorbar(rbc, true_data[-len(emu.scale_bin_centers):], yerr=yerr, label = 'Data')
#plt.plot(rbc, 10**true_pred, label = 'True')
plt.plot(rbc, 10**MAP_pred, label = 'MAP')
#plt.plot(rbc, 10**pop_xi.mean(axis = 0), label = 'Pop')#, lw = 5)

plt.loglog()
plt.legend(loc='best')
plt.show();

In [ ]:
fig = plt.figure(figsize = (10,7))
chain_samples = np.random.choice(chain.shape[0], size = 100, replace = False)
for sample in chain[chain_samples]:
    param_dict = dict(zip(emu.get_param_names(), sample))
    param_dict.update(fixed_params)
    plt.plot(rbc, 10**emu.emulate_wrt_r(param_dict)[0], alpha = 0.2)
plt.errorbar(rbc, true_data[-len(emu.scale_bin_centers):], yerr=yerr, label = 'Data', color = 'k')
#plt.plot(rbc, 10**true_pred, label = 'True')
#plt.plot(rbc, 10**MAP_pred, label = 'MAP')
#plt.plot(rbc, 10**pop_xi.mean(axis = 0), label = 'Pop')#, lw = 5)

plt.loglog()
plt.legend(loc='best')
plt.show();

In [ ]:
fig = plt.figure(figsize = (10,7))
plt.errorbar(rbc, np.zeros_like(true_data[-len(emu.scale_bin_centers):]), yerr=yerr/true_data[-len(emu.scale_bin_centers):], label = 'Data')
#plt.plot(rbc, (10**true_pred-true_data[-len(emu.scale_bin_centers):])/true_data[-len(emu.scale_bin_centers):], label = 'True')
plt.plot(rbc, (10**MAP_pred-true_data[-len(emu.scale_bin_centers):])/true_data[-len(emu.scale_bin_centers):], label = 'MAP')
plt.xscale('log')
plt.legend(loc='best')
plt.show();

In [ ]:
emu.get_param_names()

In [ ]:
fig = plt.figure(figsize=(10,7))

pred = emu.emulate_wrt_r(true_param_dict, r_bin_centers=emu.scale_bin_centers)[0]
plt.plot(rbc, 10**pred,label = 'Emu at Truth', color ='k')
#plt.plot(rbc, 10**pop_xi.mean(axis = 0), label = 'Sim' )
plt.errorbar(rbc, true_data[-len(rbc):], yerr=yerr, label = 'Data')
plt.loglog()
plt.legend(loc='best')
plt.show();

In [ ]:
fig = plt.figure(figsize=(10,7))

pred = emu.emulate_wrt_r(true_param_dict)[0]
plt.plot(rbc, 10**pred/true_data[-len(rbc):],label = 'Emu at Truth', color ='k')
#plt.plot(rbc, 10**pop_xi.mean(axis = 0), label = 'Sim' )
plt.errorbar(rbc, np.ones_like(rbc), yerr=yerr/true_data[-len(rbc):], label = 'Data')
plt.xscale('log')
plt.legend(loc='best')
plt.show();

In [ ]:
N = 100
cmap = sns.color_palette("BrBG_d", N)

In [ ]:
emu.get_param_names()

In [ ]:
true_param_dict

In [ ]:
fig = plt.figure(figsize=(10,7))
varied_pname = 'w0'
lower, upper = emu.get_param_bounds(varied_pname)
#lower, upper = 0.11, 0.12
for c, val in zip(cmap, np.linspace(lower, upper, N) ):
    param_dict = true_param_dict.copy()
    #param_dict = MAP_dict.copy()

    param_dict[varied_pname] = val
    #h_factor = (val/100)**3
    #h_factor = (100/val)
    h_factor = 1
    pred = emu.emulate_wrt_r(param_dict)[0]
    plt.plot(rbc, h_factor*(10**pred), alpha = 0.5,label = val, color =c)
    
plt.errorbar(rbc, true_data[-len(rbc):], yerr=yerr, label = 'Data')
plt.loglog()
#plt.legend(loc='best')
plt.show();

In [ ]:
fig = plt.figure(figsize=(10,7))
#varied_pname = 'Neff'
#lower, upper = emu.get_param_bounds(varied_pname)
i = 0
for c, val in zip(cmap, np.linspace(lower, upper, N) ):
    #print i, val
    i+=1
    param_dict = true_param_dict.copy()
    #param_dict = MAP_dict.copy()
    #param_dict = high_om_MAP_dict.copy()
    param_dict[varied_pname] = val
    #h_factor = (val/100)**3
    #h_factor = 1#(100/val)
    h_factor =1
    pred = emu.emulate_wrt_r(param_dict)[0]
    plt.plot(rbc, ((10**pred)*h_factor-true_data[-len(emu.scale_bin_centers):])/true_data[-len(emu.scale_bin_centers):],\
             alpha = 0.5,label = val, color =c)
    
pred = emu.emulate_wrt_r(true_param_dict)[0]
true_h_factor = 1#(true_param_dict['H0']/100)**3
#true_h_factor = (100/true_param_dict['H0'])
plt.plot(rbc, ((10**pred)*true_h_factor-true_data[-len(emu.scale_bin_centers):])/true_data[-len(emu.scale_bin_centers):], label = 'Truth', color = 'k')    
plt.plot(rbc, np.zeros_like(true_data[-len(emu.scale_bin_centers):]),  alpha = 0.3, ls = ':')
#plt.loglog()
plt.xscale('log')
#plt.legend(loc='best')
plt.show();

In [ ]:
bin = 2
#emu._kernels[bin][0].linear.variances

In [ ]:
emu._kernels[bin][0].rbf.lengthscale

In [ ]:
x = emu._x1_std*emu.x1+emu._x1_mean

In [ ]:
emu.get_param_names()

In [ ]:
np.argmin(abs(x[:,1]- 0.11386))

In [ ]:
plt.plot(emu.scale_bin_centers, emu.y[:, 4, 100]+emu._y_mean)
plt.xscale('log')

In [ ]:
x1_idx, x2_idx = 10, 100
param_dict = dict(zip(emu.get_param_names(), np.hstack([emu.x1[x1_idx]*emu._x1_std+emu._x1_mean,\
                                                        emu.x2[x2_idx]*emu._x2_std+emu._x2_mean])))

In [ ]:
param_dict

In [ ]:
data = emu.y[:, x1_idx, x2_idx]+emu._y_mean
#plt.plot(emu.scale_bin_centers, )
plt.plot(emu.scale_bin_centers, emu.emulate_wrt_r(param_dict)[0]/data)
plt.errorbar(emu.scale_bin_centers, np.ones_like(data))#, yerr=yerr/data)
plt.xscale('log')